# LT5-LE7 SR Spectral Indices

Having compared Landsat 5 (LT5) and Landsat 7 (LE7) NDVI values in sidelap areas (after Ju & Masek 2016), I found that they two platforms agree almost perfectly (linear regression through the origin yields a slope of 0.999). So, here, I use all available LT5 and LE7 images of adequate quality to build a median-value reflectance composite prior to transforming to NDVI.

> Ju, J., and J. G. Masek. 2016. The vegetation greenness trend in Canada and US Alaska from 1984-2012 Landsat data. Remote Sensing of Environment 176:1–16.

In [1]:
import ee
from IPython.display import Image
# Example iamge display:
# image = ee.Image('srtm90_v4')
# Image(url=image.getThumbUrl({'min':0, 'max': 3000}))

ee.Initialize()

# Detroit MSA bounds
detroit_msa = ee.FeatureCollection('ft:1QZFRMLeORVfc9sYNOOKD2WyOlEyI8gmj-g3CdTlH', 'geometry')

# Water mask
water = ee.Image(1).where(ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('max_extent'), ee.Image(0))

# No other way, apparently, to get a range as a list; can't map type coercion client-side...
combined_dates = ee.List([
    '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
    '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'
])

# Look at 2013, 2013, and 2015 CDL and choose 1 where pixel is always non-cultivated, zero otherwise
# Cultivated = 2 in the "cultivated" band
cdl = ee.ImageCollection('USDA/NASS/CDL')\
    .filterBounds(detroit_msa)\
    .filter(ee.Filter([
        ee.Filter.inList('system:index', ee.List(['2013', '2014', '2015']))
    ]))\
    .map(lambda img: img.select('cultivated').lt(2))\
    .reduce(ee.Reducer.allNonZero())\
    .clip(ee.Feature(detroit_msa.union().toList(1).get(0)).bounds().geometry())

## Functions

In [2]:
def annual_composite_median(collection, year):
    '''
    Creates an annual composite, given the total collection of images
    and the year in which a subset of them should be composited.
    '''
    return ee.ImageCollection(collection.filterMetadata('year', 'equals', year))\
        .median()\
        .set({'year': year})


def annual_composite_maximum(collection, year):
    '''
    Creates an annual composite, given the total collection of images
    and the year in which a subset of them should be composited.
    NOTE: For creating a maximum NDVI composite.
    '''
    return ee.ImageCollection(collection.filterMetadata('year', 'equals', year))\
        .max()\
        .set({'year': year})

    
def cloud_mask(img):
    '''
    Applies a cloud mask to an image based on the QA band of the
    Landsat data.
    '''
    # Bits 3 and 5 are cloud shadow and cloud, respectively
    cloud_shadow_bit_mask = ee.Number(2).pow(3).int()
    clouds_bit_mask = ee.Number(2).pow(5).int()
    water_mask = ee.Number(2).pow(2).int() # Bit 2 is water

    # Get the pixel QA band
    qa = img.select('pixel_qa')

    # Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)\
        .And(qa.bitwiseAnd(clouds_bit_mask).eq(0))\
        .And(qa.bitwiseAnd(water_mask).eq(0))\
        .reduceNeighborhood(ee.Reducer.anyNonZero(), ee.Kernel.square(1))
    # NOTE: Also mask those pixels that are adjacent to cloud, cloud shadow, or water

    # Return the masked image, scaled to [0, 1]
    return img.updateMask(mask).divide(10000)\
        .select(
            ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'],
            ['blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa'])\
        .set({
            'date': img.get('SENSING_TIME'),
            'year': ee.String(img.get('SENSING_TIME')).slice(0, 4)
        })


def ndvi(img):
    '''
    Calculates the normalized difference vegetation index (NDVI).
    '''
    return img.normalizedDifference(['NIR', 'red'])\
        .set({'year': img.get('year'), 'variable': 'NDVI'})

## Assets

In [3]:
lt5_collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR')\
    .filterDate('1995-01-01', '2015-12-31')\
    .filter(ee.Filter.dayOfYear(121, 273))\
    .filterMetadata('CLOUD_COVER_LAND', 'less_than', 80)\
    .filterBounds(detroit_msa)\
    .map(cloud_mask)\
    .map(lambda img: img.select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2'))

le7_collection = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR')\
    .filterDate('1995-01-01', '2015-12-31')\
    .filter(ee.Filter.dayOfYear(121, 273))\
    .filterMetadata('CLOUD_COVER_LAND', 'less_than', 80)\
    .filterBounds(detroit_msa)\
    .map(cloud_mask)\
    .map(lambda img: img.select('blue', 'green', 'red', 'NIR', 'SWIR1', 'SWIR2'))

lt5_le7_collection = ee.ImageCollection(lt5_collection).merge(le7_collection)
lt5_le7_ndvi = ee.ImageCollection(lt5_le7_collection).map(ndvi)

## Workspace

In [ ]:
# Compute the max vegetation abundance in each band, in each pixel
composites = ee.ImageCollection(combined_dates
    .map(lambda year: annual_composite_median(lt5_le7_collection, year)))\
    .map(lambda img: img.clip(detroit_msa))\
    .map(ndvi)

composites_ndvi_first = ee.ImageCollection(combined_dates
    .map(lambda year: annual_composite_maximum(lt5_le7_collection, year)))\
    .map(lambda img: img.clip(detroit_msa))

for i in xrange(0, ee.ImageCollection(composites_ndvi_first).size().getInfo()):
    img = ee.Image(ee.ImageCollection(composites_ndvi_first).toList(1, i).get(0))
    
    # Mask out permanent water, recent cultivated areas
    img_masked = img.updateMask(water).updateMask(cdl)
    
    year = list(range(1995, 2016))[i]
    task = ee.batch.Export.image.toDrive(image = img_masked, 
        description = 'LT5-LE7_SR_Maximum_NDVI_%d' % year, 
        region = detroit_msa.geometry().bounds().getInfo()['coordinates'],
        folder = 'EarthEngine', scale = 30, crs = 'EPSG:32617')
    task.start()